In [ ]:
import networkx as nx
import pandas as pd
from functools import reduce
from swiplserver import PrologMQI, PrologThread
import sys
import os
import glob
import re
from itertools import chain, combinations, product

class Target:
    def __init__(self,pr,ab):
        self.present = pr
        self.absent = ab
    
    def __str__(self):
        return f"present: {self.present} absent: {self.absent}"           
targets = []

In [ ]:
# SPECIFICATION OF THE PATTERN OF PATIENT PROFILES TO BE ANALYSED
# POSSIBLE VALUES FOR EACH FEATURE: "ANY","YES","NO"
patients_config = {
    "afib": ("kafib","YES"),
    "has_fib": ("khf","YES"),
    "heart_rate": ("khr","NO"),
    "consensus_acei": ("kcons","ANY"),
    "over75": ("kageA","ANY"),  # over75 and below55 will not be generated together (controlled later)
    "below55": ("kageB","ANY"),
    "diabete": ("kdiabete","NO"),
    "doac_int": ("kdoacint","NO"),
    "hyper": ("khyper","YES"),
    "origin": ("korigin","NO")
}

# CHOOSE ONE OF THE FOLLOWING THREE TARGETS
targets.append(Target(["major"],[]))
#targets.append(Target(["moderate"],["major"]))
#targets.append(Target(["minor"],["moderate","major"]))

In [ ]:
# MODEL SPECIFIC PARAMETERS FOR spec-AFib-hyper
patient_params = set(["afib","has_fib","heart_rate","consensus_acei","over75","below55","diabete","doac_int","hyper","origin"])
adm_drugs = set(["cbb","diltiazem","verapamil","nsbb","propranolol","carvedilol","sbb","bisoprolol","atenolol","flecainide","warfarin","doac","apixaban","dabigatran","vkant","acei","benazepril","captopril","arb","olmesortan","irbesartan","td","indapamide","chlorothiazide"])
tests_results = set(["doac_ok","doac_fail"])
CONTEXT_SIZE = 14

In [ ]:
# AUXILIARY FUNCTIONS
def state2context(s):
    state = set(s.split(','))
    context = ""
    if "afib" in state:
        context += "kafib"
    else:
        context += "empty"
    if "has_fib" in state:
        context += ",khf"
    else:
        context += ",empty"
    if "heart_rate" in state:
        context += ",khr"
    else:
        context += ",empty"
    if "consensus_acei" in state:
        context += ",kcons"
    else:
        context += ",empty"
    if "over75" in state:
        context += ",kageA"
    else:
        context += ",empty"
    if "below55" in state:
        context += ",kageB"
    else:
        context += ",empty"
    if "diabete" in state:
        context += ",kdiabete"
    else:
        context += ",empty"
    if "doac_int" in state:
        context += ",kdoacint"
    else:
        context += ",empty"
    if "hyper" in state:
        context += ",khyper"
    else:
        context += ",empty"
    if "origin" in state:
        context += ",korigin"
    else:
        context += ",empty"
    return context

def state2patient(s):
    return ';'.join(list(set(re.split(';|,',s)) & patient_params))

def state2tests(s):
    return ';'.join(list(set(re.split(';|,',s)) & tests_results))

def state2patient_with_tests(s):
    return ';'.join(list(set(re.split(';|,',s)) & (patient_params | tests_results)))

def state2drugs(s):
    return ';'.join(list(set(re.split(';|,',s)) & adm_drugs))

In [ ]:
#dotfile = nx.nx_pydot.read_dot("AFIB3.dot")
#dotfile = nx.nx_pydot.read_dot("AFIB-HYP3.dot")

print ("PATIENTS CONFIGURATION:")
patient_config_list = []
for k, v in patients_config.items():
    print (v)
    if v[1]=="YES":
        patient_config_list.append([v[0]])
    elif v[1]=="NO":
        patient_config_list.append(["empty"])
    elif v[1]=="ANY":
        patient_config_list.append([v[0],"empty"])
    else:
        raise Exception("Parse Error")

patients_list = []
for specific_patient in product(patient_config_list[0],
                                patient_config_list[1],
                                patient_config_list[2],
                                patient_config_list[3],
                                patient_config_list[4],
                                patient_config_list[5],
                                patient_config_list[6],
                                patient_config_list[7],
                                patient_config_list[8],
                                patient_config_list[9]):
    if ("kageA" in specific_patient) and ("kageB" in specific_patient):
        pass
    else:
        patients_list.append(specific_patient)

print()
print("PATIENTS TO BE ANALYZED: " + str(len(patients_list)))
print()

pat_count = 0
pat_tot = len(patients_list)
G = nx.DiGraph()
for specific_patient in patients_list:
    pat_count += 1
    specific_patient_string = ','.join(specific_patient)
    print()
    print("ANALYZING[" + str(pat_count) + "/" + str(pat_tot) + "]: " + specific_patient_string)
    param_file = open("Bioresolve_guards/params.pl",'w')
    param_file.write('mycontext("[eafib1,eafib2,eafib3,ghyper,' + specific_patient_string + ',k_doac]").\n\n')
    #eafib1,eafib2,eafib3,ghyper,kafib,khf,khr,kcons,kage,kdiabete,kdoacint,khyper,korigin,k_doac
    #param_file.write('mycontext("[e12,e13,e14,g12,' + specific_patient_string + ',k_doac]").\n\n')
    #param_file.write('mymonitor("[ m0 ]").\n\n')
    #param_file.write('mymondef("[ m0 = ([{' + state + '} inW].no({' + prolog_target +'}) + [-({' + state + '} inW)].m0) ]").\n')
    param_file.flush()
    param_file.close()
        
    with PrologMQI() as mqi:
        with mqi.create_thread() as prolog_thread:
            prolog_thread.query('set_prolog_stack(global, limit(16 000 000 000))')
            prolog_thread.query('["Bioresolve_guards/BioReSolveGuards.pl"].')
            result = prolog_thread.query('main_name("dotfile.dot").')
            print(result)
            
            dotfile = nx.nx_pydot.read_dot("dotfile.dot")
            digraph_dotfile = nx.DiGraph(dotfile)
            G = nx.compose(digraph_dotfile,G)

In [ ]:
nx.nx_pydot.write_dot(G,"AFIB-HYP-generated.dot")

In [ ]:
#G = nx.DiGraph(dotfile)

In [ ]:
#nx.draw(G, node_size=2, arrowsize=1, width=0.2)
#nx.draw_circular(G, node_size=2, arrowsize=1, width=0.2)
#nx.draw_kamada_kawai(G, node_size=2, arrowsize=1, width=0.2)
#nx.draw_shell(G, node_size=2, arrowsize=1, width=0.2)
#nx.draw_kamada_kawai(digraph_dotfile, node_size=2, arrowsize=1, width=0.2)

In [ ]:
len(list(G.nodes))

In [ ]:
# checks whether a node is in an attractor
def check_node(node):
    try:
        cycle = list(nx.find_cycle(G,node))
        tmp = map(lambda x : x[0]==node or x[1]==node, cycle) 
        return reduce(lambda b1, b2: b1 or b2, tmp)
    except:
        return false  # workaround for graphs with deadlock node (should never happen)

# compute the list of entities that are present in the attractor reachable form "node"
def compute_attractor(node):
    cycle = list(nx.find_cycle(G,node))
    tmp1 = map(lambda x: x[0].split(';') + x[1].split(';'), cycle)
    tmp2 = reduce(lambda x, y: x+y,tmp1)
    res = list(dict.fromkeys(tmp2))
    return res

In [ ]:
def count_states (d):
    tmp = 0
    for v in d.values():
        tmp += len(v)
    return tmp

# finds computations (LTS traces) that lead to the "target"
def target_computations(G,target):
    all_nodes = list(G.nodes)
    filtered = [k for k in all_nodes if check_node(k)] # filters out intermediate nodes (not in attractor)
    attractors_map = dict()
    for f in filtered:
        attractors_map[f] = compute_attractor(f) # creates a map "state -> attractor"
    for s in target.present:
        filtered = [k for k in filtered if s in attractors_map[k]]   # filters out states that do not contain s (present in the target)
    for s in target.absent:
        filtered = [k for k in filtered if s not in attractors_map[k]] # filters out state that contain s (absent in the target)
    
    # filters out states in target attractors, but that do not contain any entity in target.present
    # (slicing analysis - not yet implemented - would give an empty result on these states)
    # NOTE: this only if target.present is not empty, otherwise filtered2 would be empty as well
    if (len(target.present)>0):
        filtered2 = [k for k in filtered if len(list(set(k.split(';')) & set(target.present)))>0]
    else:
        filtered2 = filtered
    
    # cleans the attractors_map from states not in target (this step is not really necessary...)
    keys_to_delete = list() 
    for k in attractors_map.keys():
        if k not in filtered: keys_to_delete.append(k) 
    for k in keys_to_delete:
        del attractors_map[k] 
    
    patients = list()               # list of the patients that lead to the target
    patients_with_tests = list()    # list of the patients that lead to the target distinguished also by tests results
    patients_dict = {}              # for each patient, lists the states in the corresponding attractor
    
    for f in filtered2:
        next_patient = state2patient(f).split(';')
        next_patient_with_tests = state2patient_with_tests(f).split(';')
        pure_state = (f.split(';'))[CONTEXT_SIZE:] # the first CONTEXT_SIZE entities are the context
        if (not next_patient in patients):
            patients.append(next_patient)
        if (not next_patient_with_tests in patients_with_tests):
            patients_with_tests.append(next_patient_with_tests)
            patients_dict[','.join(next_patient_with_tests)] = [','.join(pure_state)]
        else:
            patients_dict[','.join(next_patient_with_tests)].append(','.join(pure_state))
    print("TARGET --> " + str(target))
    print("found " + str(len(patients)) + " patients that lead to the target")
    print("found " + str(len(patients_with_tests)) + " patients (distinguished also by tests results) that lead to the target")
    print("found " + str(count_states(patients_dict) + (len(filtered)-len(filtered2))) + " states in reachable attractors")
    print("of which " + str(count_states(patients_dict)) + " with entities in the target")
    print("NOTE: at the moment I can't distinguish between n different attractors for the same patient,")
    print("      and a single attractor of size n")
    print()
    return patients, patients_with_tests, patients_dict       


In [ ]:
FILE_OUTPUT = False

target_tot = len(targets)
target_count = 0

if FILE_OUTPUT:
    print("Writing to file...: " + "attractors_for_" + str(target.present) + "--" + str(target.absent) + ".txt")
    output_file = open("attractors_for_" + str(target.present) + "--" + str(target.absent) + ".txt","w")
    output_file.write("TO BE ANALYZED: " + str(target_tot) + " target \n \n")
else:
    print("TO BE ANALYZED: " + str(target_tot) + " target")
    print()

for target in targets:
    target_count = target_count+1
    if FILE_OUTPUT:
        output_file.write("TARGET COUNT: " + str(target_count) + "/" + str(target_tot) + "\n")
    else:
        print("TARGET COUNT: " + str(target_count) + "/" + str(target_tot))
        
    patients, patients_with_tests, patients_dict = target_computations(G,target)

    prolog_target = ','.join(target.present)

    cont = 1
    tot = str(count_states(patients_dict))
    union_set = set()
    intersection_set = set()
    first_time = True
    for ptn in patients_with_tests:
        prolog_context = state2context(','.join(ptn))
        prolog_target_states = patients_dict[','.join(ptn)]
        for i,state in enumerate(prolog_target_states):
            if FILE_OUTPUT:
                output_file.write("TEST CASE: " + str(cont) + "/" + str(tot) + "\n")
                cont=cont+1
                output_file.write("PATIENT: " + state2patient(state) + "      ATTRACTOR STATE: " + str(i+1) + "/" + str(len(prolog_target_states))  + "\n")
                output_file.write("DRUG TESTS: " + state2tests(state) + "\n")
                output_file.write("CONTEXT: " + state2context(state) + "\n")
                output_file.write("ADMINISTERED DRUGS: " + state2drugs(state) + "\n")
                output_file.write("WHOLE STATE: " + state + "\n \n")
            else:
                print("TEST CASE: " + str(cont) + "/" + str(tot))
                cont=cont+1
                print("PATIENT: " + state2patient(state) + "      ATTRACTOR STATE: " + str(i+1) + "/" + str(len(prolog_target_states)))
                print("DRUG TESTS: " + state2tests(state))
                print("CONTEXT: " + state2context(state))
                print("ADMINISTERED DRUGS: " + state2drugs(state))
                print("WHOLE STATE: " + state)
                print()

if FILE_OUTPUT:
    output_file.close()
    print("DONE")

In [ ]:
import logicmin

def append_test(ctx,lis):
    tr = list(tests_results)
    if tr[0] in lis:
        ctx.append(tr[0])
    else:
        ctx.append("empty")
    if tr[1] in lis:
        ctx.append(tr[1])
    else:
        ctx.append("empty")
    # ctx += list(set(lis) & tests_results)
    return ctx

for target in targets:
    patients, patients_with_tests, patients_dict = target_computations(G,target)
    if (len(patients)>0):
        contexts = list(map(lambda lis: append_test(state2context(','.join(lis)).split(','),lis),patients_with_tests))
        contexts2 = list(map(lambda lis: list(map(lambda x: 0 if x=='empty' else 1,lis)),contexts))    
        df = pd.DataFrame(contexts2)
        df.columns = ["afib","has_fib","heart_rate","consensus_acei","over75","below55","diabete","doac_int","hyper","origin","doac_fail","doac_ok"]
        #df = tmp.sort_values(by=[5,4,3,2,1,0],ignore_index=True)
        print("CONTEXTS THAT LEAD TO THE TARGET:")
        print(df)
        df.to_csv("contexts_to_" + str(target.present) + "--" + str(target.absent) + ".csv",index=False,header=True)
        
        print()
        # truth table 3 inputs, 2 outputs
        t = logicmin.TT(len(df.columns),1);
        for row in contexts2:
            t.add(''.join(list(map(lambda i:str(i),row))),"1")
        sols = t.solve()
        print(sols.printN(xnames=df.columns,ynames=['target'], info=True, syntax="VHDL"))
        logic_file = open("logic_formula_for_" + str(target.present) + "--" + str(target.absent) + ".txt","w")
        logic_file.write(sols.printN(xnames=df.columns,ynames=['target'], info=True, syntax="VHDL"))
        logic_file.close()

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df